Load data set into spark dataframe:

In [68]:
import os
import sys
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import size
from pandas import DataFrame
from IPython.core.interactiveshell import InteractiveShell
import matplotlib
InteractiveShell.ast_node_interactivity = "all"

# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

spark = init_spark()
filename_data = './data/endomondoHR.json'
filename_metadata = './data/endomondoMeta.json'
df = spark.read.json(filename_data, mode="DROPMALFORMED")
df_meta = spark.read.json(filename_metadata, mode="DROPMALFORMED")
type(df)

pyspark.sql.dataframe.DataFrame

Shows columns & types:

In [29]:
print('Columns & datatypes:')
dataframe = DataFrame(df.dtypes, columns =['Column Name','Data type'])
dataframe

Columns:


,Column Name,Data type
0,altitude,array<double>
1,gender,string
2,heart_rate,array<bigint>
3,id,bigint
4,latitude,array<double>
5,longitude,array<double>
6,speed,array<double>
7,sport,string
8,timestamp,array<bigint>
9,url,string


First description about dataset (this describe function exlcude the columns with list/array)

In [18]:
print('Data describe:')
df.describe().toPandas()
#print('metadata describe:')
#print(df_meta.describe().show())


Data describe:


,summary,gender,id,sport,url,userId
0,count,253020,253020,253020,253020,253020
1,mean,None,3.566244412926132E8,None,None,4619648.939783417
2,stddev,None,1.574845634895318E8,None,None,3932877.7296880507
3,min,female,99296,aerobics,https://www.endomondo.com/users/10014612/worko...,69
4,max,unknown,674008008,yoga,https://www.endomondo.com/users/9991401/workou...,15481421


In [17]:
print('Data fisrt 2 rows:')
df.limit(2).toPandas()
#print('metadata fisrt:')
#print(df_meta.first())

Data fisrt:


,altitude,gender,heart_rate,id,latitude,longitude,speed,sport,timestamp,url,userId
0,"[41.6, 40.6, 40.6, 38.4, 37.0, 34.0, 34.0, 34....",male,"[100, 111, 120, 119, 120, 116, 125, 128, 131, ...",396826535,"[60.173348765820265, 60.173239801079035, 60.17...","[24.64977040886879, 24.65014273300767, 24.6509...","[6.8652, 16.4736, 19.1988, 20.4804, 31.3956, 3...",bike,"[1408898746, 1408898754, 1408898765, 140889877...",https://www.endomondo.com/users/10921915/worko...,10921915
1,"[38.4, 39.0, 39.0, 38.2, 36.8, 36.8, 36.8, 35....",male,"[100, 105, 111, 110, 108, 115, 126, 130, 132, ...",392337038,"[60.173247596248984, 60.17320962622762, 60.172...","[24.649855233728886, 24.65015547350049, 24.650...","[9.0792, 13.284, 15.9336, 10.9476, 16.1676, 30...",bike,"[1408221682, 1408221687, 1408221699, 140822170...",https://www.endomondo.com/users/10921915/worko...,10921915


Summary on users and activities

In [40]:
userCount = df.select('userId').distinct().count()
activities = df.select('sport').distinct().toPandas()
activityCount = activities.count()
#TO DO: average workouts by user
counts = DataFrame([[userCount, activityCount]], columns= ['User Count', 'Activity types'])
print('Users and activity counts:')
counts
print('\nActivities:')
activities

Users and activity counts:


,User Count,Activity types
0,1104,sport 49 dtype: int64



Activities:


,sport
0,weight training
1,windsurfing
2,gymnastics
3,beach volleyball
4,mountain bike
5,pilates
6,cross-country skiing
7,hiking
8,kayaking
9,downhill skiing


In [71]:
df = df.withColumn('PerWorkoutRecordCount', size(col('timestamp')))

df.select('userId', 'id', 'sport','PerWorkoutRecordCount')\
   .distinct().orderBy(['userId', 'id']).limit(20).toPandas()


,userId,id,sport,PerWorkoutRecordCount
0,69,20253450,bike,499
1,69,20253456,run,500
2,69,24888428,run,499
3,69,24888434,run,500
4,69,27730553,run,499
5,69,28639037,run,276
6,69,30369221,run,500
7,69,32334585,run,499
8,69,32946380,run,95
9,69,34543589,run,25
